# 월 집계 데이터 적재
생성한 채널 집계 데이터를 적재 
1 개월 1000만 세톱 데이터에 대해 차감 연산 실행한 결과  
 - 전체 : 49 초  
 - 최초 : 28 초  
 - 재실행 : 11 초  
> 추가로 컬럼을 6배 늘려 6개월 데이터에 대해 차감한 경우 테스트 해 본다.  

In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = f'spark-02-load-inven'
DB_NAME = 'inven'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '4g')\
    .config('spark.num.executors', '4')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '2g').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [4]:
spark = spark_creation()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [5]:
# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = f'inven/table-set-1000'
file_format = 'parquet'
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("inv_rate", FloatType())
        , StructField("inv_val", LongType())
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [6]:
# %%time 
# # 기록한 파일 다시 읽어 들이기 
# # 저장 결과 확인하기 
# lines = spark.read.format(file_format).option('path', tbl_setop_name).load()
# data_count = lines.count()
# print(f'DATA Count : {data_count:,}')
# lines.show(5)

In [7]:
# %%time 
# # sparkSQL로 전체 데이터 읽어서 카운트  
# spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
# spark.sql("select count(1) from setop_view").show()

In [7]:
%%time 
# sparkSQL로 전체 데이터 읽어서 카운트  
# 건 수 차이 없는 방식. 최초, 재실행 여부 차이.  
# 최초 : 15 초 , 재실행 : 10 초   
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
# 원본 데이터에 청약량에 1000 을 강제 할당.  
spark.sql("select *, 1000 req from setop_view").createOrReplaceTempView('setop_org')
# 특정 데이터와 차감할 건수 정보 강제 생성   
# 2백만 
spark.sql("select setop, -1000 req from setop_view where setop like 'ST_J%' or setop like 'ST_A%'").createOrReplaceTempView('setop_minus')
# 전체 
#spark.sql("select setop, -1 req from setop_view").createOrReplaceTempView('setop_minus')
# 차감 요청 정보를 원본에 반영  
spark.sql("select setop, sum(req) req from (select setop, req from setop_org UNION ALL select setop, req from setop_minus) as A group by setop")\
    .createOrReplaceTempView('result')

# 계산 결과와 원본 비교  
spark.sql("select count(1) cnt, sum(req) from setop_org").show()
spark.sql("select count(1) cnt, sum(req) from result").show()

+--------+-----------+
|     cnt|   sum(req)|
+--------+-----------+
|10000000|10000000000|
+--------+-----------+

+--------+----------+
|     cnt|  sum(req)|
+--------+----------+
|10000000|8000000000|
+--------+----------+

CPU times: user 13.7 ms, sys: 814 µs, total: 14.5 ms
Wall time: 10.6 s


In [9]:
spark.stop()